# Create Functions for Model

## Create Pre-Processing Function

In [ ]:
def preprocess_data(merged_df, constraints=None):
    if constraints and 'unacceptable_pairs' in constraints:
        for item_id, factory_id in constraints['unacceptable_pairs']:
            merged_df = merged_df[~((merged_df['item_id'] == item_id) & (merged_df['factory_id'] == factory_id))]
    return merged_df


# Break Main Function Apart

## Part 1 - Select Winning Supplier

In [ ]:
def select_winning_supplier(merged_df, factory_demand_df):
    selected_suppliers = []

    for _, factory in factory_demand_df.iterrows():
        factory_id = factory['factory_id']
        factory_country = factory['factory_country']

        merged_df['local_for_local'] = merged_df['vendor_country'] == factory_country
        sorted_suppliers = merged_df.sort_values(by=['local_for_local', 'cost_per_unit'], ascending=[False, True])

        if not sorted_suppliers.empty:
            winning_supplier = sorted_suppliers.iloc[0]
            selected_suppliers.append({
                'factory_id': factory_id,
                'vendor_id': winning_supplier['vendor_id'],
                'cost_per_unit': winning_supplier['cost_per_unit'],
                'local_for_local': winning_supplier['local_for_local']
            })

    return pd.DataFrame(selected_suppliers)


## Part 2 - Allocate To Winning Factories

In [ ]:
def allocate_to_factories(selected_suppliers, target_volume_df, factory_demand_df):
    allocation_result = []

    for _, supplier in selected_suppliers.iterrows():
        vendor_id = supplier['vendor_id']
        factory_id = supplier['factory_id']

        target_volume_data = target_volume_df[target_volume_df['vendor_id'] == vendor_id]
        factory_data = factory_demand_df[factory_demand_df['factory_id'] == factory_id]

        if not target_volume_data.empty and not factory_data.empty:
            target_allocation = target_volume_data['target_allocation_pct'].iloc[0] * factory_data['VOLUME'].iloc[0]

            allocation_result.append({
                'factory_id': factory_id,
                'vendor_id': vendor_id,
                'allocated_units': target_allocation
            })

    return pd.DataFrame(allocation_result)
